In [1]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gurobipy as gp
from gurobipy import *
import pandas as pd

In [3]:
# Create sets
I = 20 #make a list of staff
J = 7 #make a list of day
K = 3 #make a list of shift

# Create a new model
model = gp.Model("staff_assignment")

# Create variables
x = model.addVars(I, J, K, vtype=GRB.BINARY, name="Staff i is assigned to day j shift k")

# Set objective
model.setObjective(gp.quicksum(x[i, j, k] for i in range(I) for j in range(J) for k in range(K)), GRB.MINIMIZE)

# Set Constraints
demand = [[6,4,3],[5,5,5],[6,4,4],[5,5,5],[5,5,3],[6,5,4],[5,5,4]]
model.addConstrs((gp.quicksum(x[i, j, k] for i in range(I)) >= demand[j][k] for j in range(J) for k in range(K)), name="demand")
model.addConstrs((gp.quicksum(x[i, j, k] for k in range(K)) <= 1 for i in range(I)  for j in range(J)), name="only work 1 shift a day")
model.addConstrs((gp.quicksum(x[i, j, k] for j in range(J) for k in range(K)) <= 5 for i in range(I)), name="only work 5 days a week")

# Optimize model
model.optimize()

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 181 rows, 420 columns and 1260 nonzeros
Model fingerprint: 0xffd36137
Variable types: 0 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 99.0000000
Presolve time: 0.00s
Presolved: 181 rows, 420 columns, 1260 nonzeros
Variable types: 0 continuous, 420 integer (420 binary)

Root relaxation: cutoff, 193 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

In [4]:
if model.status == GRB.OPTIMAL:
    # Prepare data for the table
    table_data = []

    for i in range(I):
        for j in range(J):
            shifts = [k + 1 for k in range(K) if x[i, j, k].x > 1e-6]
            if shifts:
                for k in shifts:
                    table_data.append({
                        'Staff': i + 1,  # Staff numbers starting from 1
                        'Day': j + 1,    # Days starting from 1
                        'Shift': k,      # Shifts starting from 1
                        'Status': 'Assigned'
                    })
            else:
                # No shifts assigned on this day, so staff is on leave
                table_data.append({
                    'Staff': i + 1,
                    'Day': j + 1,
                    'Shift': 'N/A',
                    'Status': 'Leave'
                })

    # Create DataFrame
    df = pd.DataFrame(table_data)

    # Print the DataFrame
    print(df)

    # Optionally, save to a CSV file
    df.to_csv('staff_assignment.csv', index=False)

else:
    print("No optimal solution found.")

     Staff  Day Shift    Status
0        1    1   N/A     Leave
1        1    2     2  Assigned
2        1    3     2  Assigned
3        1    4     2  Assigned
4        1    5   N/A     Leave
..     ...  ...   ...       ...
135     20    3     1  Assigned
136     20    4     2  Assigned
137     20    5     3  Assigned
138     20    6   N/A     Leave
139     20    7   N/A     Leave

[140 rows x 4 columns]
